In [199]:
from arcexplore import *
import polars as pl

# Creation of Input/Output complete Dataframe

In [200]:
rows = [(id,k,d,tt,it.input.tolist(), it.output.tolist()) for k,d,id,tt,_,it in ARC_DATA.data_iter()]
complete_scheme = ("ID","Source", "Set Type","Task Type","Input","Output")
Source = pl.Categorical(["ARC-AGI-1","ARC-AGI-2"])
SetType = pl.Categorical(["training","evaluation"])
TaskType = pl.Categorical(["train","test"])
complete_frame = pl.DataFrame(rows,
                              schema = complete_scheme,
                              schema_overrides = {
                                  "Source": Source,
                                  "Set Type": SetType,
                                  "Task Type": TaskType,
                                  "Input": pl.List(pl.List(pl.Int8)),
                                  "Output": pl.List(pl.List(pl.Int8))
                              },
                              orient="row")

display(complete_frame)

ID,Source,Set Type,Task Type,Input,Output
str,cat,cat,cat,list[list[i8]],list[list[i8]]
"""007bbfb7""","""ARC-AGI-1""","""training""","""train""","[[0, 7, 7], [7, 7, 7], [0, 7, 7]]","[[0, 0, … 7], [0, 0, … 7], … [0, 0, … 7]]"
"""007bbfb7""","""ARC-AGI-1""","""training""","""train""","[[4, 0, 4], [0, 0, 0], [0, 4, 0]]","[[4, 0, … 4], [0, 0, … 0], … [0, 0, … 0]]"
"""007bbfb7""","""ARC-AGI-1""","""training""","""train""","[[0, 0, 0], [0, 0, 2], [2, 0, 2]]","[[0, 0, … 0], [0, 0, … 0], … [2, 0, … 2]]"
"""007bbfb7""","""ARC-AGI-1""","""training""","""train""","[[6, 6, 0], [6, 0, 0], [0, 6, 6]]","[[6, 6, … 0], [6, 0, … 0], … [0, 0, … 6]]"
"""007bbfb7""","""ARC-AGI-1""","""training""","""train""","[[2, 2, 2], [0, 0, 0], [0, 2, 2]]","[[2, 2, … 2], [0, 0, … 0], … [0, 0, … 2]]"
…,…,…,…,…,…
"""faa9f03d""","""ARC-AGI-2""","""evaluation""","""test""","[[0, 0, … 0], [0, 0, … 0], … [0, 0, … 0]]","[[0, 0, … 0], [0, 0, … 0], … [0, 0, … 0]]"
"""fc7cae8d""","""ARC-AGI-2""","""evaluation""","""train""","[[4, 4, … 4], [6, 0, … 5], … [6, 0, … 5]]","[[1, 1, … 1], [1, 3, … 1], … [1, 1, … 1]]"
"""fc7cae8d""","""ARC-AGI-2""","""evaluation""","""train""","[[5, 5, … 1], [0, 0, … 1], … [3, 3, … 3]]","[[2, 2, … 2], [2, 8, … 2], … [2, 2, … 2]]"


# Counting Tasks

In [201]:
def level_agg(frame, cols, op_expr = None):
    op_expr = None or pl.len().alias("Count")
    for i in range(len(cols),0,-1):
        level_frame = frame.group_by(cols[:i]).agg(op_expr)
        display(level_frame)

level_agg(complete_frame.drop(("Task Type","Input","Output")).unique(),("Source", "Set Type"))

Source,Set Type,Count
cat,cat,u32
"""ARC-AGI-1""","""evaluation""",400
"""ARC-AGI-2""","""evaluation""",120
"""ARC-AGI-2""","""training""",1000
"""ARC-AGI-1""","""training""",400


Source,Count
cat,u32
"""ARC-AGI-2""",1120
"""ARC-AGI-1""",800


# Counting Grid Pairs

In [202]:
level_agg(complete_frame,("Source", "Set Type","ID","Task Type"))

Source,Set Type,ID,Task Type,Count
cat,cat,str,cat,u32
"""ARC-AGI-1""","""evaluation""","""b7f8a4d8""","""test""",1
"""ARC-AGI-2""","""training""","""53b68214""","""test""",2
"""ARC-AGI-2""","""evaluation""","""71e489b6""","""train""",3
"""ARC-AGI-1""","""training""","""4938f0c2""","""train""",3
"""ARC-AGI-2""","""training""","""d631b094""","""test""",1
…,…,…,…,…
"""ARC-AGI-2""","""training""","""bd14c3bf""","""train""",3
"""ARC-AGI-1""","""training""","""7447852a""","""train""",3
"""ARC-AGI-2""","""training""","""5ad4f10b""","""train""",3


Source,Set Type,ID,Count
cat,cat,str,u32
"""ARC-AGI-2""","""training""","""a1aa0c1e""",4
"""ARC-AGI-2""","""training""","""9b4c17c4""",6
"""ARC-AGI-2""","""training""","""52364a65""",3
"""ARC-AGI-2""","""training""","""f25fbde4""",4
"""ARC-AGI-2""","""training""","""beb8660c""",4
…,…,…,…
"""ARC-AGI-2""","""training""","""cad67732""",4
"""ARC-AGI-1""","""training""","""ddf7fa4f""",4
"""ARC-AGI-2""","""evaluation""","""fc7cae8d""",4


Source,Set Type,Count
cat,cat,u32
"""ARC-AGI-1""","""training""",1718
"""ARC-AGI-2""","""training""",4308
"""ARC-AGI-1""","""evaluation""",1782
"""ARC-AGI-2""","""evaluation""",529


Source,Count
cat,u32
"""ARC-AGI-2""",4837
"""ARC-AGI-1""",3500


# Counting Grid Cells

In [203]:
size_grid = lambda col: pl.col(col).list.len()*pl.col(col).list.get(0).list.len()
cell_col = (size_grid("Input")+size_grid("Output")).alias("Cells")
cells_frame = (complete_frame.with_columns(cell_col).
               group_by(("ID", "Source", "Set Type")).
               agg(pl.col("Cells").sum()))


level_agg(cells_frame,("Source","Set Type","ID"),pl.col("Cells").sum())

Source,Set Type,ID,Count
cat,cat,str,u32
"""ARC-AGI-2""","""evaluation""","""195c6913""",1
"""ARC-AGI-1""","""training""","""f15e1fac""",1
"""ARC-AGI-1""","""evaluation""","""9a4bb226""",1
"""ARC-AGI-2""","""evaluation""","""136b0064""",1
"""ARC-AGI-2""","""training""","""e4941b18""",1
…,…,…,…
"""ARC-AGI-2""","""training""","""ea959feb""",1
"""ARC-AGI-1""","""evaluation""","""281123b4""",1
"""ARC-AGI-2""","""training""","""d2abd087""",1


Source,Set Type,Count
cat,cat,u32
"""ARC-AGI-1""","""training""",400
"""ARC-AGI-2""","""training""",1000
"""ARC-AGI-1""","""evaluation""",400
"""ARC-AGI-2""","""evaluation""",120


Source,Count
cat,u32
"""ARC-AGI-2""",1120
"""ARC-AGI-1""",800


# Strong Size Equality

In [207]:
is_equal_expr = (size_grid("Input") == size_grid("Output")).alias("Equal")
all_equal_expr = (pl.col("Equal").all()).alias("Equal Grids")

strong_equal = (complete_frame
                .with_columns(is_equal_expr)
                .group_by(("ID", "Source", "Set Type"))
                .agg(all_equal_expr)
                .filter(pl.col("Equal Grids"))
                .drop(pl.col("Equal Grids")))

level_agg(strong_equal,("Source", "Set Type","ID"))

Source,Set Type,ID,Count
cat,cat,str,u32
"""ARC-AGI-2""","""training""","""54dc2872""",1
"""ARC-AGI-1""","""evaluation""","""4f537728""",1
"""ARC-AGI-1""","""training""","""e21d9049""",1
"""ARC-AGI-1""","""evaluation""","""55059096""",1
"""ARC-AGI-2""","""training""","""db7260a4""",1
…,…,…,…
"""ARC-AGI-1""","""training""","""bd4472b8""",1
"""ARC-AGI-2""","""training""","""88a10436""",1
"""ARC-AGI-2""","""training""","""776ffc46""",1


Source,Set Type,Count
cat,cat,u32
"""ARC-AGI-2""","""evaluation""",82
"""ARC-AGI-1""","""evaluation""",270
"""ARC-AGI-1""","""training""",262
"""ARC-AGI-2""","""training""",680


Source,Count
cat,u32
"""ARC-AGI-2""",762
"""ARC-AGI-1""",532
